In [201]:
# 1. ke každému článku v kategoriích Narození v... najdeme celkové pageviews a vybereme po kategorii page s max pageviews
import json

In [115]:
# requesty na obecné wikipedia API a na REST API pro pageviews
import requests
session = requests.Session()
def main_api_request(request):
    api_url = "https://cs.wikipedia.org/w/api.php"
    header = {"Api-User-Agent": "michal.zlatkovsky@rozhlas.cz"}
    reply = session.get(url=api_url, headers=header, params=request)
    return reply.json()

def pageview_api_request(page):
    api_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/cs.wikipedia/all-access/all-agents/" + page + "/monthly/2015070100/2019070100"
    header = {"Api-User-Agent": "michal.zlatkovsky@rozhlas.cz"}
    reply = session.get(url=api_url, headers=header)
    return reply.json()

In [128]:
# součet pageviews z měsíčních dat
def total_pageviews(data): 
    return sum([month["views"] for month in data["items"]])

In [109]:
# Kategorie:Narození_v_Česku_podle_krajů -> subcats -> články subcats -> pageviews -> max

In [110]:
# krajské kategorie
request = {
    "action": "query",
    "format": "json",
    "list": "categorymembers",
    "cmtitle": "Kategorie:Narození v Česku podle krajů",
    "cmlimit": 500
}

data = main_api_request(request)
kraje_cat = [member["title"] for member in data["query"]["categorymembers"]]

In [111]:
# obsah krajských kategorií; ns 14 je Kategorie:, tím se vyloučí pages zařazené do krajů
kraje_subcat = {}
for kraj in kraje_cat:
    request["cmtitle"] = kraj
    kraj_data = main_api_request(request)
    kraje_subcat[kraj] = {member["title"]: {} for member in kraj_data["query"]["categorymembers"] if member["ns"] == 14}
# Praha je město i kraj - vohek
kraje_subcat["Kategorie:Narození v Praze"] = {"Kategorie:Narození v Praze": {}}

In [ ]:
# obsah obecních kategorií; ns 0 jsou běžné články; + práce s limitem
count = 0
for kraj in kraje_subcat:
    for obec_cat in kraje_subcat[kraj]:
        request["cmtitle"] = obec_cat
        if "cmcontinue" in request: del request["cmcontinue"]
        obec_rodaci = {}
        finished = False
        while not finished:
            obec_data = main_api_request(request)
            obec_rodaci.update({member["title"]: 0 for member in obec_data["query"]["categorymembers"] if member["ns"] == 0})
            if "continue" in obec_data: 
                request["cmcontinue"] = obec_data["continue"]["cmcontinue"]
            else:
                finished = True
        kraje_subcat[kraj][obec_cat].update(obec_rodaci)
        count += 1
        print("finished " + str(count))

In [ ]:
# zjištění pageviews - 18814 rodáků
count = 0
for kraj in kraje_subcat:
    for obec in kraje_subcat[kraj]:
        for rodak in kraje_subcat[kraj][obec]:
            count += 1
            if kraje_subcat[kraj][obec][rodak] == 0:
                try:
                    kraje_subcat[kraj][obec][rodak] = total_pageviews(pageview_api_request(rodak))
                except:
                    kraje_subcat[kraj][obec][rodak] = 0
                print("finished " + str(count))

In [140]:
# uložení všech dat pro jistotu
with open("complete_data.json", "w", encoding="utf8") as file:
    file.write(json.dumps(kraje_subcat))

In [ ]:
# vyfiltrování "nejslavnějšího" rodáka pro každou obecní kategorii
# try/except pro případ, že obecní kat prázdná
kraje_subcat_filtered = {}
for kraj in kraje_subcat:
    kraje_subcat_filtered[kraj] = {}
    for obec in kraje_subcat[kraj]:
        rodaci = kraje_subcat[kraj][obec]
        try:
            kraje_subcat_filtered[kraj][obec] = sorted(rodaci.items(), key=lambda x: x[1], reverse=True)[0]
        except:
            pass

In [183]:
with open("filtered_data.json", "w", encoding="utf8") as file:
    file.write(json.dumps(kraje_subcat_filtered))

In [200]:
# krajské celebrity
for kraj in kraje_subcat_filtered:
    krajsti_rodaci = [kraje_subcat_filtered[kraj][obec] for obec in kraje_subcat_filtered[kraj]]
    top_rodak = sorted(krajsti_rodaci, key=lambda x: x[1], reverse=True)[0]
    print(kraj[10:] + ":", top_rodak[0], "-", top_rodak[1])

Narození v Jihočeském kraji: Jan Hus - 846945
Narození v Jihomoravském kraji: Tomáš Garrigue Masaryk - 1286380
Narození v Karlovarském kraji: Jaromír Soukup - 438214
Narození v Královéhradeckém kraji: Karel Čapek - 903156
Narození v Libereckém kraji: Vlasta Burian - 396573
Narození v Moravskoslezském kraji: Jiří Drahoš - 680510
Narození v Olomouckém kraji: Ondřej Sokol - 310925
Narození v Pardubickém kraji: Bedřich Smetana - 607220
Narození v Plzeňském kraji: Karel Gott - 875349
Narození v Praze: Karel IV. - 2117348
Narození ve Středočeském kraji: Miloš Zeman - 1060359
Narození v Ústeckém kraji: Karolína Plíšková - 327401
Narození v Kraji Vysočina: Martina Sáblíková - 363405
Narození ve Zlínském kraji: Karel Kryl - 340033
